In [1]:
!pip install boto3

     -------------------------------------- 139.3/139.3 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 82.2/82.2 kB 4.5 MB/s eta 0:00:00
     --------------------------------------- 12.3/12.3 MB 10.6 MB/s eta 0:00:00


Note that you will need to configure AWS CLI manually for your logins to AWS before running the following code.

# Helper functions:

In [2]:
import boto3
import json
from botocore.exceptions import ClientError

regions = [
    'us-east-1',  # US East (N. Virginia)
    'us-east-2',  # US East (Ohio)
    'us-west-1',  # US West (N. California)
    'us-west-2',  # US West (Oregon)
    'ap-south-1',  # Asia Pacific (Mumbai)
    'ap-northeast-3',  # Asia Pacific (Osaka)
    'ap-northeast-2',  # Asia Pacific (Seoul)
    'ap-southeast-1',  # Asia Pacific (Singapore)
    'ap-southeast-2',  # Asia Pacific (Sydney)
    'ap-northeast-1',  # Asia Pacific (Tokyo)
    'ca-central-1',  # Canada (Central)
    'eu-central-1',  # Europe (Frankfurt)
    'eu-west-1',  # Europe (Ireland)
    'eu-west-2',  # Europe (London)
    'eu-west-3',  # Europe (Paris)
    'eu-north-1',  # Europe (Stockholm)
    'sa-east-1'   # South America (São Paulo)
]

#helper functions
def get_amazon_linux_2023_ami(region):
    try:
        ec2_client = boto3.client('ec2', region_name=region)
        response = ec2_client.describe_images(
            Filters=[
                {
                    'Name': 'description',
                    'Values': ['Amazon Linux 2023 AMI*']
                },
                {
                    'Name': 'architecture',
                    'Values': ['x86_64']
                }
            ], 
            Owners=['amazon']
            # Sorting by creation date to get the latest AMI
        )
        if 'Images' in response and len(response['Images']) > 0:
            # Assuming the latest image is the first in the list after sorting by creation date
            latest_image = sorted(response['Images'], key=lambda x: x['CreationDate'], reverse=True)[0]
            return latest_image['ImageId']
        else:
            return None
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None


def create_security_group(vpc, ec2):
    try:
        # Check for existing security groups with the same name in the VPC
        existing_sgs = ec2.security_groups.filter(
            Filters=[
                {'Name': 'vpc-id', 'Values': [vpc.id]},
                {'Name': 'group-name', 'Values': ['allow-all-traffic']}
            ]
        )
        duplicated_sgs = [sg for sg in existing_sgs]
        
        if duplicated_sgs:
            print("Duplicated security groups found:")
            for sg in duplicated_sgs:
                print(f" - {sg.group_id} ({sg.group_name})")
            return duplicated_sgs
        
        # Create a new security group if no duplicates found
        sg = ec2.create_security_group(
            Description='Allow all inbound and outbound traffic',
            GroupName='allow-all-traffic',
            VpcId=vpc.id
        )
        try:
            sg.authorize_ingress(
                IpPermissions=[{
                    'IpProtocol': '-1',
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }]
            )
        except ClientError as e:
            print(f"An error occurred when setting ingress: {e}")
        try:
            sg.authorize_egress(
                IpPermissions=[{
                    'IpProtocol': '-1',
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }]
            )
        except ClientError as e:
            print(f"An error occurred when setting egress: {e}")
        return sg
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None
    


def create_vpc(ec2, CidrBlock, region_suffix):
    try:
        vpc = ec2.create_vpc(CidrBlock=CidrBlock)
        vpc.wait_until_available()
        vpc.create_tags(Tags=[{"Key": "Name", "Value": f"VPC_waveSpreader_experiment_{region_suffix}"}])
        return vpc
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def create_subnet(vpc, ec2, cidr_block):
    try:
        subnet = ec2.create_subnet(CidrBlock=cidr_block, VpcId=vpc.id)
        subnet.create_tags(Tags=[{"Key": "Name", "Value": f"Subnet_waveSpreader_experiment_{vpc.id}"}])
        return subnet
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def create_internet_gateway(vpc, ec2):
    try:
        igw = ec2.create_internet_gateway()
        vpc.attach_internet_gateway(InternetGatewayId=igw.id)
        return igw
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def create_route_table(vpc, igw):
    try:
        route_table = vpc.create_route_table()
        route_table.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=igw.id)
        return route_table
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def associate_route_table(route_table, subnet):
    try:
        route_table.associate_with_subnet(SubnetId=subnet.id)
    except ClientError as e:
        print(f"An error occurred: {e}")

# Function to create a VPC peering connection
def create_vpc_peering_connection(ec2, vpc_id, peer_vpc_id, peer_region):
    try:
        peering_connection = ec2.create_vpc_peering_connection(
            VpcId=vpc_id,
            PeerVpcId=peer_vpc_id,
            PeerRegion=peer_region
        )
        if 'VpcPeeringConnection' in peering_connection:
            return peering_connection['VpcPeeringConnection']
        else:
            print("Peering connection creation failed.")
            return None
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

# Function to accept a VPC peering connection
def accept_vpc_peering_connection(ec2_resource, peering_connection_id):
    try:
        peering_connection = ec2_resource.VpcPeeringConnection(peering_connection_id)
    except ClientError as e:
        print(f"An error occurred: {e}")
    try:
        peering_connection.accept()
    except ClientError as e:
        print(f"An error occurred: {e}")


def update_route_tables_for_peering(route_table, peering_connection_id, peer_cidr):
    try:
        route_table.create_route(
            DestinationCidrBlock=peer_cidr,
            VpcPeeringConnectionId=peering_connection_id
        )
    except ClientError as e:
        print(f"An error occurred: {e}")
        
def delete_internet_gateways(ec2):
    try:
        for igw in ec2.internet_gateways.all():
            for attachment in igw.attachments:
                igw.detach_from_vpc(VpcId=attachment['VpcId'])
            igw.delete()
            print(f"Deleted Internet Gateway: {igw.id}")
    except ClientError as e:
        print(f"An error occurred while deleting Internet Gateway: {e}")

def delete_route_tables(ec2):
    try:
        for rt in ec2.route_tables.all():
            if not rt.associations_attribute:
                rt.delete()
                print(f"Deleted Route Table: {rt.id}")
    except ClientError as e:
        print(f"An error occurred while deleting Route Table: {e}")

def delete_vpc_peering_connections(ec2):
    try:
        for pcx in ec2.vpc_peering_connections.all():
            if pcx.status['Code'] != 'deleting':
                pcx.delete()
                print(f"Deleted VPC Peering Connection: {pcx.id}")
    except ClientError as e:
        print(f"An error occurred while deleting VPC Peering Connection: {e}")

def delete_user_managed_prefix_lists(ec2_client):
    try:
        prefix_lists = ec2_client.describe_managed_prefix_lists()['PrefixLists']
        for pl in prefix_lists:
            if not pl['PrefixListName'].startswith('com.amazonaws'):
                ec2_client.delete_managed_prefix_list(PrefixListId=pl['PrefixListId'])
                print(f"Deleted Managed Prefix List: {pl['PrefixListId']}")
    except ClientError as e:
        print(f"An error occurred while deleting Managed Prefix List: {e}")
        
def delete_instances(instance_ids,default_region='eu-north-1'):
  # Specify a default region to create the initial client
    try:
        # Create a default EC2 client to get the instance details
        ec2_client = boto3.client('ec2', region_name=default_region)
        # Describe the instances to get their details
        response = ec2_client.describe_instances(InstanceIds=instance_ids)
        logger.info(f"Describe instances response: {response}")
        
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                instance_id = instance['InstanceId']
                availability_zone = instance['Placement']['AvailabilityZone']
                region = get_region_from_az(availability_zone)
                #logger.info(f"Instance {instance_id} is in availability zone {availability_zone}, region {region}")
                
                if region:
                    try:
                        # Create an EC2 client for the specific region
                        ec2 = boto3.client('ec2', region_name=region)
                        # Terminate the instance
                        ec2.terminate_instances(InstanceIds=[instance_id])
                        logger.info(f"Terminated instance {instance_id} in region {region}")
                    except Exception as e:
                        logger.error(f"Failed to terminate instance {instance_id}: {str(e)}")
                else:
                    logger.error(f"Could not determine a valid region from availability zone {availability_zone} for instance {instance_id}")
    except Exception as e:
        logger.error(f"Failed to describe instances: {str(e)}")

def delete_vpc(ec2, vpc_id):
    try:
        vpc = ec2.Vpc(vpc_id)
        
        # Detach and delete internet gateways
        for igw in vpc.internet_gateways.all():
            vpc.detach_internet_gateway(InternetGatewayId=igw.id)
            igw.delete()
        
        # Delete subnets
        for subnet in vpc.subnets.all():
            subnet.delete()
        
        # Delete route tables
        for rt in vpc.route_tables.all():
            if not rt.associations_attribute:
                rt.delete()
        
        # Delete security groups
        for sg in vpc.security_groups.all():
            if sg.group_name != 'default':
                sg.delete()
        
        # Delete VPC peering connections
        for pcx in ec2.vpc_peering_connections.all():
            if pcx.requester_vpc.id == vpc_id or pcx.accepter_vpc.id == vpc_id:
                pcx.delete()
        
        # Finally, delete the VPC
        vpc.delete()
        print(f"Deleted VPC: {vpc_id}")
    except ClientError as e:
        print(f"An error occurred: {e}")
        
def create_instance(ec2, ami_id, subnet_id, private_ip, security_group_id, user_data):
    try:
        instance = ec2.create_instances(
            ImageId=ami_id,
            InstanceType='t3.micro',
            MinCount=1,
            MaxCount=1,
            NetworkInterfaces=[{
                'DeviceIndex': 0,
                'AssociatePublicIpAddress': True,
                'SubnetId': subnet_id,
                'PrivateIpAddress': private_ip,
                'Groups': [security_group_id],
            }],
            IamInstanceProfile={
                'Name': 'Experiment_'  # Replace with the name of your IAM role
            },
            UserData=user_data
        )[0]
        return instance
    except ClientError as e:
        logger.error(f"Error creating instance: {str(e)}")
        return None
def delete_vpc_dependencies(vpc_id,region):
    ec2 = boto3.client('ec2',region_name=region)
    
    # Delete Subnets
    subnets = ec2.describe_subnets(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['Subnets']
    for subnet in subnets:
        ec2.delete_subnet(SubnetId=subnet['SubnetId'])
        print(f"Deleted Subnet: {subnet['SubnetId']}")

    # Delete Route Tables
    route_tables = ec2.describe_route_tables(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['RouteTables']
    for route_table in route_tables:
        if not route_table['Associations'][0]['Main']:  # Skip the main route table
            ec2.delete_route_table(RouteTableId=route_table['RouteTableId'])
            print(f"Deleted Route Table: {route_table['RouteTableId']}")

    # Delete Network Interfaces
    network_interfaces = ec2.describe_network_interfaces(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['NetworkInterfaces']
    for interface in network_interfaces:
        ec2.delete_network_interface(NetworkInterfaceId=interface['NetworkInterfaceId'])
        print(f"Deleted Network Interface: {interface['NetworkInterfaceId']}")

    # Delete Internet Gateways
    internet_gateways = ec2.describe_internet_gateways(Filters=[{'Name': 'attachment.vpc-id', 'Values': [vpc_id]}])['InternetGateways']
    for gateway in internet_gateways:
        ec2.detach_internet_gateway(InternetGatewayId=gateway['InternetGatewayId'], VpcId=vpc_id)
        ec2.delete_internet_gateway(InternetGatewayId=gateway['InternetGatewayId'])
        print(f"Deleted Internet Gateway: {gateway['InternetGatewayId']}")

    # Delete NAT Gateways
    nat_gateways = ec2.describe_nat_gateways(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['NatGateways']
    for nat_gateway in nat_gateways:
        ec2.delete_nat_gateway(NatGatewayId=nat_gateway['NatGatewayId'])
        print(f"Deleted NAT Gateway: {nat_gateway['NatGatewayId']}")

    # Delete Security Groups
    security_groups = ec2.describe_security_groups(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['SecurityGroups']
    for security_group in security_groups:
        if security_group['GroupName'] != 'default':  # Skip the default security group
            ec2.delete_security_group(GroupId=security_group['GroupId'])
            print(f"Deleted Security Group: {security_group['GroupId']}")

    # Delete VPN Connections and Gateways
    vpn_connections = ec2.describe_vpn_connections(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['VpnConnections']
    for vpn_connection in vpn_connections:
        ec2.delete_vpn_connection(VpnConnectionId=vpn_connection['VpnConnectionId'])
        print(f"Deleted VPN Connection: {vpn_connection['VpnConnectionId']}")

    vpn_gateways = ec2.describe_vpn_gateways(Filters=[{'Name': 'attachment.vpc-id', 'Values': [vpc_id]}])['VpnGateways']
    for vpn_gateway in vpn_gateways:
        ec2.delete_vpn_gateway(VpnGatewayId=vpn_gateway['VpnGatewayId'])
        print(f"Deleted VPN Gateway: {vpn_gateway['VpnGatewayId']}")

    # Delete Endpoints
    endpoints = ec2.describe_vpc_endpoints(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['VpcEndpoints']
    for endpoint in endpoints:
        ec2.delete_vpc_endpoints(VpcEndpointIds=[endpoint['VpcEndpointId']])
        print(f"Deleted VPC Endpoint: {endpoint['VpcEndpointId']}")

    # Delete Peering Connections
    peering_connections = ec2.describe_vpc_peering_connections(Filters=[{'Name': 'requester-vpc-info.vpc-id', 'Values': [vpc_id]}])['VpcPeeringConnections']
    for peering_connection in peering_connections:
        ec2.delete_vpc_peering_connection(VpcPeeringConnectionId=peering_connection['VpcPeeringConnectionId'])
        print(f"Deleted VPC Peering Connection: {peering_connection['VpcPeeringConnectionId']}")

    # Finally, delete the VPC
    ec2.delete_vpc(VpcId=vpc_id)
    print(f"Deleted VPC: {vpc_id}")
def delete_vpcs():
    for region in regions:
        print (region)
        ec2 = boto3.resource('ec2', region_name=region)
        ec2client = boto3.client('ec2', region_name=region)
        # Retrieve all VPCs
        vpcs = ec2.vpcs.all()

        for vpc in vpcs:
            # Delete VPC peering connections
            peering_connections = ec2client.describe_vpc_peering_connections()['VpcPeeringConnections']
            for peering in peering_connections:
                peering_id = peering['VpcPeeringConnectionId']
                peering_state = peering['Status']['Code']
                try:
                    ec2client.delete_vpc_peering_connection(VpcPeeringConnectionId=peering_id)
                except ClientError as e:
                        print(f'Failed to delete vpc_peering_connection  {peering_id}: {e}')
            # Detach and delete internet gateways
            for igw in vpc.internet_gateways.all():
                print(f'Detaching and deleting internet gateway {igw.id}')
                igw.detach_from_vpc(VpcId=vpc.id)
                igw.delete()

            # Delete route table associations
            for rt in vpc.route_tables.all():
                for association in rt.associations:
                    if not association.main:
                        print(f'Deleting route table association {association.id}')
                        association.delete()
                if not rt.associations:
                    print(f'Deleting route table {rt.id}')
                    rt.delete()

            # Delete security groups (skip the default one)
            # Delete security groups (skip the default one)
            for sg in vpc.security_groups.all():
                if sg.group_name != 'default':
                    print(f'Deleting security group {sg.id}')
                    try:
                        sg.delete()
                    except ClientError as e:
                        print(f'Failed to delete security group {sg.id}: {e}')

            # Delete network ACLs (skip the default one)
            for nacl in vpc.network_acls.all():
                if not nacl.is_default:
                    print(f'Deleting network ACL {nacl.id}')
                    try:
                        nacl.delete()
                    except ClientError as e:
                        print(f'Failed to delete network ACL {nacl.id}: {e}')
                    

            # Delete subnets
            for subnet in vpc.subnets.all():
                print(f'Deleting subnet {subnet.id}')
                try:
                    subnet.delete()
                except ClientError as e:
                    print(f'Failed to delete network ACL {subnet.id}: {e}')
                

            # Finally, delete the VPC
    for region in regions:
            print (region)
            ec2 = boto3.resource('ec2', region_name=region)
            ec2client = boto3.client('ec2', region_name=region)
            # Retrieve all VPCs
            vpcs = ec2.vpcs.all()
            try:
                vpc.delete()
            except ClientError as e:
                    print(f'Failed to delete VPC {vpc.id}: {e}')


def get_security_group_id(sg_group_dict):
    sg_group = sg_group_dict.get('sg_group')
    if sg_group:
        sg_id = sg_group.id
        return sg_id
    else:
        raise ValueError("No 'sg_group' key found in the dictionary")

# Create VPC subnets, internet_gateways over different regions:

In [5]:
vpc_info = {}
ip_base = "10.0."
subnet_size = 20 // len(regions)  # Distributing /20 subnets among 17 regions
for i, region in enumerate(regions):
    ec2 = boto3.resource('ec2', region_name=region)
    cidr_block = f"{ip_base}{i+1}.0/24"    
    # Create VPC with a unique name
    vpc = create_vpc(ec2, cidr_block, region_suffix=i)
    if not vpc:
        print(f"Failed to create VPC in {region}")
        continue
    # Create Internet Gateway
    cidr_block = f"{ip_base}{i+1}.0/25"
    subnet = create_subnet(vpc, ec2, cidr_block)
    
    if not subnet:
        continue
    
    # Create Internet Gateway
    igw = create_internet_gateway(vpc, ec2)
    if not igw:
        continue
    
    # Create Route Table and associate with Subnet
    route_table = create_route_table(vpc, igw)
    if not route_table:
        continue
    associate_route_table(route_table, subnet)
    # Create Security Group
    sg = create_security_group(vpc, ec2)
    if not sg:
        print(f"Failed to create Security Group in {region}")
    vpc_info[region] = {
        'vpc_id': vpc.id,
        'route_table_id': route_table.id,
        'cidr_block': vpc.cidr_block,
        'subnet_id': subnet.id,
        'sg_group' : sg.id
    }     

print(json.dumps(vpc_info))

An error occurred when setting egress: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupEgress operation: the specified rule "peer: 0.0.0.0/0, ALL, ALLOW" already exists
An error occurred when setting egress: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupEgress operation: the specified rule "peer: 0.0.0.0/0, ALL, ALLOW" already exists
An error occurred when setting egress: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupEgress operation: the specified rule "peer: 0.0.0.0/0, ALL, ALLOW" already exists
An error occurred when setting egress: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupEgress operation: the specified rule "peer: 0.0.0.0/0, ALL, ALLOW" already exists
An error occurred when setting egress: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupEgress operation: the specified rule "peer

Please copy the above {} to Start_test_.ipynb as vpc_info before running Start_test_.ipynb.

# Create VPC Peering Connections and update route tables

In [6]:

peering_connections = []
for i, region in enumerate(regions):
    ec2_client = boto3.client('ec2', region_name=region)
    ec2_resource = boto3.resource('ec2', region_name=region)
    vpc_id = vpc_info[region]['vpc_id']
    for j, peer_region in enumerate(regions):
        if i < j:  # Ensure that each pair is only processed once
            peer_vpc_id = vpc_info[peer_region]['vpc_id']
            peering_connection = create_vpc_peering_connection(ec2_client, vpc_id, peer_vpc_id, peer_region)
            if peering_connection:
                peering_connections.append((region, peer_region, peering_connection['VpcPeeringConnectionId']))
                
                
# Wait for all peering connections to be accepted
for region, peer_region, peering_connection_id in peering_connections:
    ec2_client = boto3.client('ec2', region_name=region)
    peering_accepted = False
    while not peering_accepted:
        try:
            peering_connection = ec2_client.describe_vpc_peering_connections(VpcPeeringConnectionIds=[peering_connection_id])
            state = peering_connection['VpcPeeringConnections'][0]['Status']['Code']
            if state == 'active':
                peering_accepted = True
            elif state == 'initiating-request':
                print(f"Waiting for peering connection {peering_connection_id} between {region} and {peer_region} to be accepted.")
                time.sleep(3)  # Wait and check again
            else:
                #print(f"Unexpected state {state} for peering connection {peering_connection_id}.")
                break
        except ClientError as e:
            print(f"An error occurred: {e}")
            break

# Accept VPC Peering Connections
for region, peer_region, peering_connection_id in peering_connections:
    accept_vpc_peering_connection(boto3.resource('ec2', region_name=peer_region), peering_connection_id)

# Update Route Tables with Peering Connections
for region, peer_region, peering_connection_id in peering_connections:
    ec2 = boto3.resource('ec2', region_name=region)
    route_table_id = vpc_info[region]['route_table_id']
    route_table = ec2.RouteTable(route_table_id)
    peer_cidr_block = vpc_info[peer_region]['cidr_block']
    update_route_tables_for_peering(route_table, peering_connection_id, peer_cidr_block)
    ec2 = boto3.resource('ec2', region_name=peer_region)
    route_table_id = vpc_info[peer_region]['route_table_id']
    route_table = ec2.RouteTable(route_table_id)
    peer_cidr_block = vpc_info[region]['cidr_block']
    update_route_tables_for_peering(route_table, peering_connection_id, peer_cidr_block)
    print(f"Peering connection created and accepted between {region} and {peer_region} with ID {peering_connection_id}")

print("Setup complete")

# Output the IP range of each region
for region, info in vpc_info.items():
    print(f"Region: {region}, CIDR Block: {info['cidr_block']}")

Peering connection created and accepted between us-east-1 and us-east-2 with ID pcx-036c7ed638a7c6bd4
Peering connection created and accepted between us-east-1 and us-west-1 with ID pcx-0dd987e8176d386cd
Peering connection created and accepted between us-east-1 and us-west-2 with ID pcx-0847a9906ca342670
Peering connection created and accepted between us-east-1 and ap-south-1 with ID pcx-01b09dde94eadf4e3
Peering connection created and accepted between us-east-1 and ap-northeast-3 with ID pcx-055c6469a88cb62ff
Peering connection created and accepted between us-east-1 and ap-northeast-2 with ID pcx-09aa6fc711600de63
Peering connection created and accepted between us-east-1 and ap-southeast-1 with ID pcx-0293cf2bd3976b3db
Peering connection created and accepted between us-east-1 and ap-southeast-2 with ID pcx-095c891decd7368ae
Peering connection created and accepted between us-east-1 and ap-northeast-1 with ID pcx-0496035384dcffc2a
Peering connection created and accepted between us-east-

Peering connection created and accepted between ap-northeast-3 and eu-west-3 with ID pcx-08e77ee851d699cc0
Peering connection created and accepted between ap-northeast-3 and eu-north-1 with ID pcx-0849f358c1070f7e7
Peering connection created and accepted between ap-northeast-3 and sa-east-1 with ID pcx-069e5fdef5b6c65f9
Peering connection created and accepted between ap-northeast-2 and ap-southeast-1 with ID pcx-02b8a8233e0897edc
Peering connection created and accepted between ap-northeast-2 and ap-southeast-2 with ID pcx-0273c25d26396ecbf
Peering connection created and accepted between ap-northeast-2 and ap-northeast-1 with ID pcx-044c4e7ed51ee105b
Peering connection created and accepted between ap-northeast-2 and ca-central-1 with ID pcx-09f8464fb6ec5a53d
Peering connection created and accepted between ap-northeast-2 and eu-central-1 with ID pcx-08eeac2675822b2dc
Peering connection created and accepted between ap-northeast-2 and eu-west-1 with ID pcx-0747b72d908187988
Peering connect

# Delete the testbed

After running the experiment, you may want to delete the environment. 
Please run the following two cells multiple times manually to ensure that all dependencies are removed and the subnets deleted.

In [5]:
for i in range (3):
  #  for region in regions:
   #     ec2 = boto3.resource('ec2', region_name=region)
    #    delete_vpc (ec2, vpc_info[region]['vpc_id'])
    delete_vpcs()
    for region in regions:
        print (region)
        ec2 = boto3.resource('ec2', region_name=region)
        ec2_client = boto3.client('ec2', region_name=region)
        delete_internet_gateways(ec2)
        delete_route_tables(ec2)
        delete_user_managed_prefix_lists(ec2_client)
        delete_vpc_peering_connections(ec2)
    for region in regions:
        for vpc in ec2.vpcs.all():
            try:
                delete_vpc_dependencies(vpc.id,region)
                vpc.delete()
            except ClientError as e:
                print(f"error{e}")
                continue
    delete_vpcs()

us-east-1
Detaching and deleting internet gateway igw-08bee6fb2d742c3e0
Deleting route table rtb-0e4d05861e562769b
Deleting security group sg-0b45c49f0d3066694
us-east-2
Detaching and deleting internet gateway igw-0b4a51d467b807909
Deleting route table association rtbassoc-06519c3fc1cb17495
Deleting security group sg-0c252ad483afa0e78
Deleting subnet subnet-0005a4b7682ed9823
us-west-1
Detaching and deleting internet gateway igw-0048dc934f7cc0463
Deleting route table association rtbassoc-05bff1fea968bce64
Deleting security group sg-0fe2d839448179c63
Deleting subnet subnet-069cbf605e573d8b2
us-west-2
Detaching and deleting internet gateway igw-03e9752ad6d63e99f
Deleting route table association rtbassoc-0e3b3ead8bd8d7524
Deleting security group sg-0d172a13ca4408f01
Deleting subnet subnet-07f323295a6580a9d
ap-south-1
Detaching and deleting internet gateway igw-0f33fa76bb68694b0
Deleting route table association rtbassoc-078f929c0b236ba8a
Deleting security group sg-0193a6f1e8a925934
Deleting

Deleted VPC Peering Connection: pcx-09aa6fc711600de63
Deleted VPC Peering Connection: pcx-0293cf2bd3976b3db
Deleted VPC Peering Connection: pcx-001a9d5ba86406f96
Deleted VPC Peering Connection: pcx-028f68f51fe40cdb9
Deleted VPC Peering Connection: pcx-0dd987e8176d386cd
Deleted VPC Peering Connection: pcx-036c7ed638a7c6bd4
Deleted VPC Peering Connection: pcx-0496035384dcffc2a
us-east-2
Deleted Route Table: rtb-0bea3b4f3996adaf2
Deleted VPC Peering Connection: pcx-07d66782c56dd7432
Deleted VPC Peering Connection: pcx-014e2a7034b499f0c
Deleted VPC Peering Connection: pcx-0251b596b973d4da7
Deleted VPC Peering Connection: pcx-0000caf321ce54a01
Deleted VPC Peering Connection: pcx-0203e72dbe3351acb
Deleted VPC Peering Connection: pcx-067495a3942a11d3f
Deleted VPC Peering Connection: pcx-0dcace9716e094e8a
Deleted VPC Peering Connection: pcx-0b3eefdd2727b604c
Deleted VPC Peering Connection: pcx-0502ae1459022b63d
Deleted VPC Peering Connection: pcx-0d4c0f1acbe0f4f2f
Deleted VPC Peering Connectio

Deleted VPC Peering Connection: pcx-00a0e97d89c452106
Deleted VPC Peering Connection: pcx-0264b610c444a16e4
Deleted VPC Peering Connection: pcx-0016f4b6bb69f80c6
Deleted VPC Peering Connection: pcx-0c8ea4afce42e5c3c
Deleted VPC Peering Connection: pcx-08509a1be1c4a2418
Deleted VPC Peering Connection: pcx-044c4e7ed51ee105b
Deleted VPC Peering Connection: pcx-062bab24991ec6ac0
Deleted VPC Peering Connection: pcx-0496035384dcffc2a
ca-central-1
Deleted Route Table: rtb-03b2cd26aa8536fff
Deleted VPC Peering Connection: pcx-06cc8aca1a2f5a2eb
Deleted VPC Peering Connection: pcx-0ea8874b71664189f
Deleted VPC Peering Connection: pcx-0832659d64792a323
Deleted VPC Peering Connection: pcx-052041cf166a7aded
Deleted VPC Peering Connection: pcx-0399fd35b8b3725ab
Deleted VPC Peering Connection: pcx-002517259b120264e
Deleted VPC Peering Connection: pcx-07fcfe02294dfad8d
Deleted VPC Peering Connection: pcx-0ae3bd8976f902ac0
Deleted VPC Peering Connection: pcx-0cc27c68632f30e45
Deleted VPC Peering Connec

errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
Deleted VPC: vpc-0a92d3c9112fe2898
errorAn error occurred (InvalidVpcID.NotFound) when calling the DeleteVpc operation: The vpc ID 'vpc-0a92d3c9112fe2898' does not exist
us-east-1
us-east-2


KeyboardInterrupt: 

In [6]:
for i in range (3):
  #  for region in regions:
   #     ec2 = boto3.resource('ec2', region_name=region)
    #    delete_vpc (ec2, vpc_info[region]['vpc_id'])
    delete_vpcs()
    for region in regions:
        print (region)
        ec2 = boto3.resource('ec2', region_name=region)
        ec2_client = boto3.client('ec2', region_name=region)
        delete_internet_gateways(ec2)
        delete_route_tables(ec2)
        delete_user_managed_prefix_lists(ec2_client)
        delete_vpc_peering_connections(ec2)
    for region in regions:
        for vpc in ec2.vpcs.all():
            try:
                delete_vpc_dependencies(vpc.id,region)
                vpc.delete()
            except ClientError as e:
                print(f"error{e}")
                continue
    delete_vpcs()

us-east-1
us-east-2
us-west-1
us-west-2
ap-south-1
ap-northeast-3
ap-northeast-2
ap-southeast-1
ap-southeast-2
ap-northeast-1
ca-central-1
eu-central-1
eu-west-1
eu-west-2
eu-west-3
eu-north-1
Deleting security group sg-0dbc8e0a2704bfd80
Failed to delete security group sg-0dbc8e0a2704bfd80: An error occurred (DependencyViolation) when calling the DeleteSecurityGroup operation: resource sg-0dbc8e0a2704bfd80 has a dependent object
Deleting subnet subnet-0d6494112066cd216
Failed to delete network ACL subnet-0d6494112066cd216: An error occurred (DependencyViolation) when calling the DeleteSubnet operation: The subnet 'subnet-0d6494112066cd216' has dependencies and cannot be deleted.
sa-east-1
us-east-1
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
us-east-2
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when

Deleted VPC Peering Connection: pcx-0e6ce5e903c686a28
Deleted VPC Peering Connection: pcx-0e96b4b281d34de48
Deleted VPC Peering Connection: pcx-0fffbad8403853014
Deleted VPC Peering Connection: pcx-0fbd8e6a1a1f79c03
Deleted VPC Peering Connection: pcx-062bab24991ec6ac0
Deleted VPC Peering Connection: pcx-0e8c8c628d4c02185
ap-northeast-3
Deleted VPC Peering Connection: pcx-07d66782c56dd7432
Deleted VPC Peering Connection: pcx-04423e7d364e5a89d
Deleted VPC Peering Connection: pcx-08e77ee851d699cc0
Deleted VPC Peering Connection: pcx-0ee8334ceaf0e4b48
Deleted VPC Peering Connection: pcx-07c2aea3630c01b59
Deleted VPC Peering Connection: pcx-03d59797644da151d
Deleted VPC Peering Connection: pcx-069e5fdef5b6c65f9
Deleted VPC Peering Connection: pcx-002517259b120264e
Deleted VPC Peering Connection: pcx-0fa4ab79759545434
Deleted VPC Peering Connection: pcx-055c6469a88cb62ff
Deleted VPC Peering Connection: pcx-0711be2ead85fbed7
Deleted VPC Peering Connection: pcx-01569b9cac9ec2873
Deleted VPC P

Deleted VPC Peering Connection: pcx-09a4a922c2910a004
Deleted VPC Peering Connection: pcx-08e77ee851d699cc0
Deleted VPC Peering Connection: pcx-0fe891b54828490cd
Deleted VPC Peering Connection: pcx-0b3a8ba8a00506c52
Deleted VPC Peering Connection: pcx-0e976e4d8dfcce1b7
Deleted VPC Peering Connection: pcx-063228507db110d6c
Deleted VPC Peering Connection: pcx-0c2e0af771fa8334c
Deleted VPC Peering Connection: pcx-05a8a31f218b3e909
Deleted VPC Peering Connection: pcx-00b54fb69b2a227ea
Deleted VPC Peering Connection: pcx-0921f6323642d185c
Deleted VPC Peering Connection: pcx-05aa24c80214857e7
Deleted VPC Peering Connection: pcx-02f4f134230c913cd
Deleted VPC Peering Connection: pcx-0bead6ada93fb0e2a
Deleted VPC Peering Connection: pcx-0520498a1c99ddd75
Deleted VPC Peering Connection: pcx-06bb19bbda56f3702
Deleted VPC Peering Connection: pcx-04c1eee62fe591e86
eu-north-1
Deleted VPC Peering Connection: pcx-010ad516d21fab89d
Deleted VPC Peering Connection: pcx-052041cf166a7aded
Deleted VPC Peeri

Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
ap-northeast-3
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
ap-northeast-2
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
ap-southeast-1
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
ap-southeast-2
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has depend

Deleted VPC Peering Connection: pcx-043aa5a43d6f5c299
Deleted VPC Peering Connection: pcx-00ebea602568d341e
Deleted VPC Peering Connection: pcx-01569b9cac9ec2873
Deleted VPC Peering Connection: pcx-037bc1b15b6eb4fb4
Deleted VPC Peering Connection: pcx-09f8464fb6ec5a53d
Deleted VPC Peering Connection: pcx-06dd9b14d928f0d78
Deleted VPC Peering Connection: pcx-0273c25d26396ecbf
Deleted VPC Peering Connection: pcx-09aa6fc711600de63
Deleted VPC Peering Connection: pcx-08eeac2675822b2dc
Deleted VPC Peering Connection: pcx-044c4e7ed51ee105b
Deleted VPC Peering Connection: pcx-0e8c8c628d4c02185
ap-southeast-1
Deleted VPC Peering Connection: pcx-09a4a922c2910a004
Deleted VPC Peering Connection: pcx-06cc8aca1a2f5a2eb
Deleted VPC Peering Connection: pcx-0251b596b973d4da7
Deleted VPC Peering Connection: pcx-02b8a8233e0897edc
Deleted VPC Peering Connection: pcx-03d59797644da151d
Deleted VPC Peering Connection: pcx-0265b95e5bab9f969
Deleted VPC Peering Connection: pcx-0fae0f3f40918be46
Deleted VPC P

Deleted VPC Peering Connection: pcx-003f02df2b4e4ff75
Deleted VPC Peering Connection: pcx-0264b610c444a16e4
Deleted VPC Peering Connection: pcx-0832861f66021ed0a
Deleted VPC Peering Connection: pcx-0849f358c1070f7e7
Deleted VPC Peering Connection: pcx-0a5bc6390816ccc2d
sa-east-1
Deleted VPC Peering Connection: pcx-0e42e3db1a277c1da
Deleted VPC Peering Connection: pcx-09be6356b979d4856
Deleted VPC Peering Connection: pcx-03b5bddf2b1c9ffb0
Deleted VPC Peering Connection: pcx-031dd5b07100723d9
Deleted VPC Peering Connection: pcx-0b3a8ba8a00506c52
Deleted VPC Peering Connection: pcx-069e5fdef5b6c65f9
Deleted VPC Peering Connection: pcx-0dcace9716e094e8a
Deleted VPC Peering Connection: pcx-0ce1320bf8cfda9fd
Deleted VPC Peering Connection: pcx-02aa307843d0a250a
Deleted VPC Peering Connection: pcx-0dca57142a5483c1d
Deleted VPC Peering Connection: pcx-0761305565b3bdfeb
Deleted VPC Peering Connection: pcx-03231fb0292288fe8
Deleted VPC Peering Connection: pcx-0262f4f16e68d8bd8
Deleted VPC Peerin

Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
eu-west-1
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
eu-west-2
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
eu-west-3
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
eu-north-1
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot b

Deleted VPC Peering Connection: pcx-04423e7d364e5a89d
Deleted VPC Peering Connection: pcx-095c891decd7368ae
Deleted VPC Peering Connection: pcx-0d9eef6229d846dd3
Deleted VPC Peering Connection: pcx-0f48406e736728045
Deleted VPC Peering Connection: pcx-093f0c91b50d31346
Deleted VPC Peering Connection: pcx-0adab1ed2ff26e23e
Deleted VPC Peering Connection: pcx-0dd994c1f95f8c0cb
Deleted VPC Peering Connection: pcx-0551cd4ff2e3b02f5
Deleted VPC Peering Connection: pcx-0962fa8256cbae7d0
Deleted VPC Peering Connection: pcx-0273c25d26396ecbf
Deleted VPC Peering Connection: pcx-0e6ce5e903c686a28
Deleted VPC Peering Connection: pcx-0bead6ada93fb0e2a
Deleted VPC Peering Connection: pcx-0ed202839e3bb4166
Deleted VPC Peering Connection: pcx-0b839252843396757
Deleted VPC Peering Connection: pcx-07280990417ecd91b
Deleted VPC Peering Connection: pcx-0a2ca6e65d1f5e1ba
ap-northeast-1
Deleted VPC Peering Connection: pcx-052fc64c19dcad8ff
Deleted VPC Peering Connection: pcx-063228507db110d6c
Deleted VPC P

Deleting subnet subnet-0d6494112066cd216
Failed to delete network ACL subnet-0d6494112066cd216: An error occurred (DependencyViolation) when calling the DeleteSubnet operation: The subnet 'subnet-0d6494112066cd216' has dependencies and cannot be deleted.
sa-east-1
us-east-1
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
us-east-2
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
us-west-1
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-09880e68e842e4bae' has dependencies and cannot be deleted.
us-west-2
Failed to delete VPC vpc-09880e68e842e4bae: An error occurred (DependencyViolation) when calling the DeleteVpc op